In [1]:
from PIL import Image
import glob
import pandas as pd

good_pop = glob.glob('../screenshots/*.png')

bad_pop = glob.glob('../screenshots/bad/*.png')
print(len(good_pop), len(bad_pop))

330


In [2]:
from collections import Counter
from itertools import product
import colorsys
def normalize_pixel(pixel, num_color_steps=16):
    factor = (256//num_color_steps)
    return tuple(factor*(int((c)/factor)) for c in pixel)

def extract_features(image, num_channel_values=4):
    pixels_array = image.load()
    w, h = image.size
    
    pixels = [normalize_pixel(pixels_array[x,y],num_channel_values) for x,y in product(range(w), range(h))]
    color_counts = Counter(pixels)
    num_pixels = len(pixels)
    pixel_freqs = dict()
    common_pixel_f = (0,0)
    for k,v in color_counts.items():
        pixel_freqs.update({str(k):v/num_pixels})
        if v > common_pixel_f[1]:
            common_pixel = (k,v)
#     return pixel_freqs
#     print(color_counts.most_common(10))
    if len(color_counts) > 1:
        major_hue = colorsys.rgb_to_hsv(*(color_counts.most_common(2)[1][0]))[0]
    else:
        major_hue = colorsys.rgb_to_hsv(*(color_counts.most_common(2)[0][0]))[0]
    return {
        'num_colors': len(color_counts),
        'major_hue': major_hue,
        'empty_proportion': color_counts.most_common(1)[0][1]/(w*h),
        
    }

def extract_file(path):
    try:
        return extract_features(Image.open(path))
    except Exception as e:
        print("Error with path={}\n{}".format(path,e))
        return None

extract_features(Image.open(good_pop[0]))

{'empty_proportion': 0.9952068865740741, 'major_hue': 0.5, 'num_colors': 14}

In [3]:
from multiprocessing import Pool
import time
pool = Pool()

In [4]:

ti = time.time()
good_set = list(pool.map(extract_file, bad_pop))
tf = time.time()
print(tf-ti)
frame = pd.DataFrame(good_set)
frame.describe()

269.40669417381287


,empty_proportion,major_hue,num_colors
count,204.000000,204.000000,204.000000
mean,0.985600,0.354847,28.245098
std,0.013208,0.306392,12.057282
min,0.944731,0.000000,1.000000
25%,0.975260,0.000000,17.000000
50%,0.986738,0.333333,29.000000
75%,0.997668,0.666667,38.000000
max,1.000000,0.916667,54.000000


In [5]:
ti = time.time()
good_set = list(pool.map(extract_file, good_pop))
tf = time.time()
print(tf-ti)
frame = pd.DataFrame(good_set)
frame.describe()

442.6141142845154


,empty_proportion,major_hue,num_colors
count,330.000000,330.000000,330.000000
mean,0.990423,0.396970,27.127273
std,0.005746,0.295453,8.338699
min,0.964265,0.000000,9.000000
25%,0.987812,0.166667,21.000000
50%,0.991405,0.333333,27.000000
75%,0.994864,0.666667,33.000000
max,0.999039,0.916667,53.000000
